In [ ]:
import os
import numpy as np
from caret_analyze import Architecture, Application, check_procedure, Lttng, LttngEventFilter
from caret_analyze.plot import Plot, message_flow, callback_graph, chain_latency, callback_sched
from bokeh.plotting import output_notebook, figure, show
import pandas as pd
import pandas_bokeh
import matplotlib.pyplot as plt
import IPython
output_notebook()

In [ ]:
observed_data_path = os.environ['HOME'] + '/.ros/tracing/caret_sample'

In [ ]:
lttng = Lttng(observed_data_path, force_conversion=True, event_filters=[LttngEventFilter.strip_filter(0.2, 0.2)])
arch = Architecture('lttng', observed_data_path)
app = Application(arch, lttng)

In [ ]:
node_list = []
target_node_name_list = []
node_count = lttng.get_count(['node_name'])

for i, item in enumerate(node_count.iterrows()):
    target_node_name_list.append(item[0])
target_node_name_list.sort()

for node_name in target_node_name_list:
    try:
        node_list.append(app.get_node(node_name))
    except:
        print(f'Failed to find node \"{node_name}\"')   # You can ignore this message

for node in node_list:
    print(node.node_name)

In [ ]:
def show_histogram(measurement):
    for item in measurement.iteritems():
        df_to_histogram = pd.DataFrame()
        if 'callback_start_timestamp' not in item[1].name[1]:
            item[1].name = item[1].name[0]
            df_to_histogram = pd.concat([df_to_histogram, item[1]], axis=1)
            df_to_histogram = df_to_histogram.dropna().astype(float)
            if not df_to_histogram.empty:
                df_to_histogram.plot_bokeh.hist()
    # df_to_histogram.hist(layout=(7, 2), figsize=(10,18))
    plt.close()

In [ ]:
# Frequency
for node in node_list:
    plot = Plot.create_callback_frequency_plot(node)
    measurement = plot.to_dataframe()
    measurement_mean = measurement.mean()
    
    print(f'------> {node.node_name} <------')
    for index, mean in zip(measurement_mean.index, measurement_mean):
        if 'frequency' in index[1]:
            print(f"{index[0]}")
            print(f"\t{index[1]}:{mean}")
    plot.show('system_time')
    # measurement.plot_bokeh.hist()

    show_histogram(measurement)
    print('------------')

In [ ]:
# Jitter
for node in node_list:
    plot = Plot.create_callback_jitter_plot(node)
    measurement = plot.to_dataframe()
    measurement_mean = measurement.mean()
    
    print(f'------> {node.node_name} <------')
    for index, mean in zip(measurement_mean.index, measurement_mean):
        if 'period' in index[1]:
            print(f"{index[0]}")
            print(f"\t{index[1]}:{mean}")
    plot.show('system_time')
    show_histogram(measurement)
    print('------------')

In [ ]:
# Latency
from caret_analyze.plot import Plot
for node in node_list:
    plot = Plot.create_callback_latency_plot(node)
    measurement = plot.to_dataframe()
    measurement_mean = measurement.mean()
    
    print(f'------> {node.node_name} <------')
    for index, mean in zip(measurement_mean.index, measurement_mean):
        if 'latency' in index[1]:
            print(f"{index[0]}")
            print(f"\t{index[1]}:{mean}")
    plot.show('system_time')
    show_histogram(measurement)
    print('------------')

In [ ]:
for node in node_list:
    print(node.node_name)
    if len(node.callbacks) < 2:
        continue
    for i, callback in enumerate(node.callbacks):
        print(f"-----{i} symbol")
        print(f"---- {callback.callback_name} -----")
        print(callback.symbol)